In [9]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [10]:
fact_bookings_df = pd.read_csv('fact_bookings.csv',)
fact_aggregated_bookings_df = pd.read_csv('fact_aggregated_bookings.csv')

In [11]:
# TO find the correlation between revenue and seasonal occupancy

# creating a dictionary to map each room to type.
rooms = {}
dim_rooms_df = pd.read_csv('dim_rooms.csv')
for id in dim_rooms_df['room_id']:
    rooms[id] = dim_rooms_df['room_class']

In [12]:
# creating a dictionary to map each date to month(seasonal occupancy)
dates = {}
dim_date_df = pd.read_csv('dim_date.csv')
for date in dim_date_df['date']:
    dates[date] = dim_date_df['mmm yy']

In [13]:
# loading the data_frame that contains the data for each hotel based on hotel_id
dim_hotels_df = pd.read_csv('dim_hotels.csv')

In [14]:
# for each month season, find the net percentage of rooms occupied vs capacity
seasonal_occupancy = {}
for i in range(len(fact_aggregated_bookings_df)):
    date = pd.to_datetime(fact_aggregated_bookings_df.loc[i, 'check_in_date'])
    month = date.month
    occupancy_percent = (fact_aggregated_bookings_df.loc[i, 'successful_bookings'] * 100) / fact_aggregated_bookings_df.loc[i, 'capacity']
    
    if month in seasonal_occupancy:
        seasonal_occupancy[month].append(occupancy_percent)
    else:
        seasonal_occupancy[month] = [occupancy_percent]
# computing the average seasonal occupancy        
average_seasonal_occupancy = {month: sum(percents)/len(percents) for month, percents in seasonal_occupancy.items()}


In [ ]:
# for each month season, find the net revenue realised.
seasonal_revenue = {}
for i in range(len(fact_bookings_df)):
    date = pd.to_datetime(fact_bookings_df.loc[i, 'check_in_date'])
    month = date.month
    revenue = fact_bookings_df.loc[i, 'revenue_realized']
    
    if month in seasonal_revenue:
        seasonal_revenue[month] += revenue
    else:
        seasonal_revenue[month] = revenue

In [ ]:
# plot the data
df_plot = pd.DataFrame({
    'Month': list(seasonal_revenue.keys()),
    'Revenue': list(seasonal_revenue.values()),
    'Occupancy (%)': [average_seasonal_occupancy[month] for month in seasonal_revenue.keys()]
})

# Optional: sort by month
df_plot = df_plot.sort_values(by='Month')

# Optional: map month number to name
month_map = {5: 'May', 6: 'Jun', 7: 'Jul'}

df_plot['Month'] = df_plot['Month'].map(month_map)

fig, ax1 = plt.subplots(figsize=(10, 6))

sns.barplot(x='Month', y='Revenue', data=df_plot, ax=ax1, color='lightblue',width = 0.3)
ax1.set_ylabel('Net Realized Revenue', color='darkblue')
ax1.tick_params(axis='y', labelcolor='darkblue')

for i, v in enumerate(df_plot['Revenue']):
    ax1.text(i, 0, f'{v:.0f}', color='blue', fontsize=9, ha='center', va='bottom')
    
ax2 = ax1.twinx()
sns.lineplot(x='Month', y='Occupancy (%)', data=df_plot, ax=ax2, color='green', marker='o')
ax2.set_ylabel('Avg. Seasonal Occupancy (%)', color='green')
ax2.tick_params(axis='y', labelcolor='green')
for x, y in zip(df_plot['Month'], df_plot['Occupancy (%)']):
    ax2.text(x, y, f'{y:.1f}%', color='black', fontsize=9, ha='center', va='bottom')
# correlation
df = pd.DataFrame({
    'Revenue': pd.Series(seasonal_revenue),
    'Occupancy': pd.Series(average_seasonal_occupancy)
})
correlation = df.corr().loc['Revenue', 'Occupancy']
ax1.text(0.22, 0.95, f'Correlation: {correlation:.2f}', transform=ax1.transAxes,
         fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round", fc="wheat", ec="black"))
plt.title('Monthly Realized Revenue vs. Average Seasonal Occupancy Percentage')
plt.tight_layout()
plt.show()
